In [1]:
!pip install pyspark


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Reviews.csv to Reviews (1).csv


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Reviews Analysis") \
    .getOrCreate()


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Reviews Analysis") \
    .getOrCreate()


In [5]:
df = spark.read.csv("Reviews.csv", header=True, inferSchema=True)


In [6]:
from pyspark.sql.functions import col, count, avg, desc

# Schema
df.printSchema()

# First 5 rows
df.show(5)

# Total rows
print("Total rows:", df.count())

# Missing values
df.select([count(col(c)).alias(c) for c in df.columns]).show()

# Average score
df.groupBy().avg("Score").show()

# Top 10 most reviewed products
df.groupBy("ProductId") \
  .agg(count("*").alias("review_count")) \
  .orderBy(desc("review_count")) \
  .show(10)

# Top 10 reviewers
df.groupBy("UserId") \
  .agg(count("*").alias("review_count")) \
  .orderBy(desc("review_count")) \
  .show(10)

# Rating distribution
df.groupBy("Score").count().orderBy("Score").show()


root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Go

AnalysisException: "Score" is not a numeric column. Aggregation function can only be applied on a numeric column.

In [7]:
from pyspark.sql.functions import col, count, avg, desc

# Convert Score to numeric type
df = df.withColumn("Score", col("Score").cast("double"))

# Schema
df.printSchema()

# First 5 rows
df.show(5)

# Total rows
print("Total rows:", df.count())

# Missing values
df.select([count(col(c)).alias(c) for c in df.columns]).show()

# Average score
df.groupBy().avg("Score").show()

# Top 10 most reviewed products
df.groupBy("ProductId") \
  .agg(count("*").alias("review_count")) \
  .orderBy(desc("review_count")) \
  .show(10)

# Top 10 reviewers
df.groupBy("UserId") \
  .agg(count("*").alias("review_count")) \
  .orderBy(desc("review_count")) \
  .show(10)

# Rating distribution
df.groupBy("Score").count().orderBy("Score").show()


root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: double (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|  5.0|1303862400|Go

In [8]:

spark.stop()

